In [1]:
from helper.loader import load_ratings, load_movies, load_lists

In [2]:
load_ratings()

,movie_id,rating_id,rating_score,user_id
0,1066,15610495,3.0,41579158
1,1066,10704606,2.0,85981819
2,1066,10177114,2.0,4208563
3,1066,10130280,3.0,9820140
4,1066,8357049,4.0,68654088
...,...,...,...,...
15459940,1065,35403,4.0,39480101
15459941,1065,27955,4.0,60235331
15459942,1065,26731,4.0,42678387
15459943,1065,25765,3.0,18576002


In [3]:
load_movies()

,movie_id,movie_title,movie_release_year,movie_url,movie_title_language,movie_popularity,movie_image_url,director_id,director_name,director_url
0,1,La Antena,2007.0,http://mubi.com/films/la-antena,en,105,https://images.mubicdn.net/images/film/1/cache...,131,Esteban Sapir,http://mubi.com/cast/esteban-sapir
1,2,Elementary Particles,2006.0,http://mubi.com/films/elementary-particles,en,23,https://images.mubicdn.net/images/film/2/cache...,73,Oskar Roehler,http://mubi.com/cast/oskar-roehler
2,3,It's Winter,2006.0,http://mubi.com/films/its-winter,en,21,https://images.mubicdn.net/images/film/3/cache...,82,Rafi Pitts,http://mubi.com/cast/rafi-pitts
3,4,Kirikou and the Wild Beasts,2005.0,http://mubi.com/films/kirikou-and-the-wild-beasts,en,46,https://images.mubicdn.net/images/film/4/cache...,"89, 90","Michel Ocelot, Bénédicte Galup","http://mubi.com/cast/michel-ocelot, http://mub..."
4,5,Padre Nuestro,2007.0,http://mubi.com/films/padre-nuestro,en,7,https://images.mubicdn.net/images/film/5/cache...,92,Christopher Zalla,http://mubi.com/cast/christopher-zalla
...,...,...,...,...,...,...,...,...,...,...
226570,263706,Kill Chain: The Cyber War on America's Elections,2020.0,http://mubi.com/films/kill-chain-the-cyber-war...,en,0,https://images.mubicdn.net/images/film/263706/...,"458986, 458987, 531718","Simon Ardizzone, Russell Michaels, Sarah Teale","http://mubi.com/cast/simon-ardizzone, http://m..."
226571,263707,We're Here,2020.0,http://mubi.com/films/we-re-here,en,0,https://images.mubicdn.net/images/film/263707/...,609756,Peter Logreco,http://mubi.com/cast/peter-logreco
226572,263708,The Weasel,1962.0,http://mubi.com/films/the-weasel,en,0,https://images.mubicdn.net/images/film/263708/...,499055,Branko Marjanović,http://mubi.com/cast/branko-marjanovic
226573,263709,Scenes from the Family Life,1931.0,http://mubi.com/films/scenes-from-the-family-life,en,0,https://images.mubicdn.net/images/film/263709/...,777386,Ivan Brlić,http://mubi.com/cast/ivan-brlic


In [4]:
load_lists()

,user_id,list_id,list_title,list_movie_number,list_update_timestamp_utc,list_creation_timestamp_utc,list_followers,list_url,list_comments,list_description,list_cover_image_url,list_first_image_url,list_second_image_url,list_third_image_url
0,85981819,1969,250 Favourite Films,250,2019-11-26 03:20:17,2009-12-18 13:04:48,23,http://mubi.com/lists/250-favourite-films,5,"<p>In a loose order, but an order nonetheless....",https://assets.mubicdn.net/images/film/115/ima...,https://assets.mubicdn.net/images/film/115/ima...,https://assets.mubicdn.net/images/film/3664/im...,https://assets.mubicdn.net/images/film/187/ima...
1,85981819,3946,Films that should be better known,181,2020-05-01 05:54:39,2010-01-30 16:27:27,308,http://mubi.com/lists/films-that-should-be-bet...,11,<p>These are the films that need to be in more...,https://assets.mubicdn.net/images/film/20780/i...,https://assets.mubicdn.net/images/film/20780/i...,https://assets.mubicdn.net/images/film/19167/i...,https://assets.mubicdn.net/images/film/23056/i...
2,85981819,6683,Films by country (completely stolen from Glemaud),179,2020-04-12 08:31:35,2010-03-31 08:55:10,32,http://mubi.com/lists/films-by-country-complet...,5,"<p>Being a compulsive listmaker, cinéphile and...",https://assets.mubicdn.net/images/film/40676/i...,https://assets.mubicdn.net/images/film/40676/i...,https://assets.mubicdn.net/images/film/99891/i...,https://assets.mubicdn.net/images/film/32597/i...
3,85981819,8865,Play it cool.,56,2018-12-14 14:53:41,2010-05-10 08:18:40,10,http://mubi.com/lists/play-it-cool,1,<p>A work in progress…</p>\n<p>This is intende...,https://assets.mubicdn.net/images/film/11573/i...,https://assets.mubicdn.net/images/film/11573/i...,https://assets.mubicdn.net/images/film/25451/i...,https://assets.mubicdn.net/images/film/1418/im...
4,85981819,13796,Cinemalism (work in progress),121,2019-11-26 03:20:23,2010-08-25 14:23:56,43,http://mubi.com/lists/cinemalism-work-in-progress,3,<p>An ongoing list collecting the best in mini...,https://assets.mubicdn.net/images/film/113749/...,https://assets.mubicdn.net/images/film/113749/...,https://assets.mubicdn.net/images/film/160533/...,https://assets.mubicdn.net/images/film/95112/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80306,77265912,4091,hong kong,1,2010-02-02 06:04:59,2010-02-02 06:04:59,0,http://mubi.com/lists/hong-kong,0,None,https://assets.mubicdn.net/images/film/226/ima...,https://assets.mubicdn.net/images/film/226/ima...,None,None
80307,74636861,3183,Movies to watch,1,2010-01-14 04:28:39,2010-01-14 04:28:39,0,http://mubi.com/lists/movies-to-watch--2,0,None,https://assets.mubicdn.net/images/film/2377/im...,https://assets.mubicdn.net/images/film/2377/im...,None,None
80308,15089938,46648,To See!,1,2012-05-08 13:45:47,2012-05-08 13:45:47,0,http://mubi.com/lists/to-see--106,0,None,https://assets.mubicdn.net/images/film/37733/i...,https://assets.mubicdn.net/images/film/37733/i...,None,None
80309,12567302,3181,my mine,1,2010-01-14 01:49:51,2010-01-14 01:49:51,0,http://mubi.com/lists/my-mine,0,<p>peluang usaha ahasu gnaulep <br>\ncheat gam...,https://assets.mubicdn.net/images/film/3405/im...,https://assets.mubicdn.net/images/film/3405/im...,None,None
